In [ ]:
import rioxarray 
import xarray
import rasterio
from rasterio.enums import Resampling
from rasterio import Affine
from rasterio.crs import CRS
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def convert_hdf_to_cog(scene_id, product = "S30"):
    
    """
    This function receives the scene_id of an HLS scene (in a format similar to "HLS.S30.T14RNS.2020005.v1.4"
    and converts the scene from HDF format to COG. 
    
    Assumptions:
    - The corresponding HDF file for the scene is located at `/data/scene_id.hdf`
    - The output will be written to `/data/scene_id/*.tif` and contains all the bands. 
    
    Inputs:
    - scene_id: The scene ID of the HLS scene
    - product: the HLS product ID. Default is S30, but it can be S30, L30, S30_ANGLES, L30_ANGLES
    
    """
    
    import os
    cmd = f"python3 /hls-hdf_to_cog/hls_hdf_to_cog/hls_hdf_to_cog.py --product {product} /data/{scene_id}.hdf --output-dir /data/{scene_id}/"
    os.system(cmd)

In [ ]:
#conversion of a sample scene_id
scene_id = "HLS.S30.T15STT.2020002.v1.4"

# pass the scene_id to the function
convert_hdf_to_cog(scene_id)

In [ ]:
def reproject_hls_to_cdl(scene_folder,
                         bands = ["B01", "B02", "B03", "B04", "B05", "B06", "B07", "B08", "B8A", "B09", "B10", "B11", "B12"],
                         cdl_file = "data/2022_30m_cdls_clipped.tif"):
    
    """
    This function receives the path to a folder that contains all GeoTIFF files (for various bands)
    of a HLS scene, and reprojects those to the target CDL CRS and grid. 
    
    Assumptions:
    - scene_folder has a file structure like: ".../<scene_id>/<scene_id>.<band_id>.tiff
    - scene_folder should not have a "/" at the end
    
    Inputs:
    - scene_folder: is the path to the folder that contains HLS GeoTIFF files for all bands of HLS
    - bands: list of bands of HLS that should be reprojected (default is all bands)
    - cdl_file: contains the path to the clipped CDL GeoTIFF file
    
    """
    
    for band in bands:
        xds = xarray.open_rasterio(f"{scene_folder}/{scene_folder.split('/')[-1]}.{band}.tif")
        cdl = xarray.open_rasterio(cdl_file)
        xds_new = xds.rio.reproject_match(cdl, resampling = Resampling.bilinear)
        xds_new.rio.to_raster(raster_path = f"{scene_folder}/{scene_folder.split('/')[-1]}.{band}.5070.tif")
        

In [ ]:
#sample reprojection of a scene
reproject_hls_to_cdl("/data/HLS.S30.T15STT.2020002.v1.4", 
                     bands = ["B03"])

In [ ]:
xds = xarray.open_rasterio("/data/HLS.S30.T15STT.2020002.v1.4/HLS.S30.T15STT.2020002.v1.4.B02.tif")

In [ ]:
cdl = xarray.open_rasterio("data/2022_30m_cdls_clipped.tif")

In [ ]:
xds_new = xds.rio.reproject_match(cdl, resampling = Resampling.bilinear)

In [ ]:
xds_new.rio.to_raster(raster_path="sample_cog_w_cdl.tif")